In [1]:
#STEP 1: Install and Authenticate
!pip install --quiet gspread google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

from google.colab import files
import os
import google.auth
from google.auth.transport.requests import Request, AuthorizedSession
import requests
import json
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import Flow

print("🔼 Upload your client_secrets.json file")
uploaded = files.upload()
uploaded_filename = list(uploaded.keys())[0]
if uploaded_filename != "client_secrets.json":
    os.rename(uploaded_filename, "client_secrets.json")
    print(f"✅ Renamed {uploaded_filename} ➜ client_secrets.json")

import google.auth.transport.requests
from google_auth_oauthlib.flow import Flow
from googleapiclient.discovery import build
import gspread

SCOPES = [
    "https://www.googleapis.com/auth/analytics.edit",
    "https://www.googleapis.com/auth/analytics.readonly",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/analytics.provision"
]

flow = Flow.from_client_secrets_file(
    "client_secrets.json",
    scopes=SCOPES,
    redirect_uri="urn:ietf:wg:oauth:2.0:oob"
)

auth_url, _ = flow.authorization_url(prompt="consent")
print("🔗 Open this link in a new tab and authorize:")
print(auth_url)

code = input("Paste the authorization code here: ")
flow.fetch_token(code=code)

creds = flow.credentials

admin = build("analyticsadmin", "v1beta", credentials=creds)
gc = gspread.authorize(creds)
print("✅ Authorized GA4 Admin API + Google Sheets successfully!")

ACKNOWLEDGEMENT_STRING = "I acknowledge that I have the necessary privacy disclosures and rights from my end users for the collection and processing of their data, including the association of such data with the visitation information Google Analytics collects from my site and/or app property."
API_BASE_URL = "https://analyticsadmin.googleapis.com/v1beta"

def acknowledge_user_data_collection(credentials, property_id: str, acknowledgement: str):
    try:
        authed_session = AuthorizedSession(credentials)

        api_url = f"{API_BASE_URL}/{property_id}:acknowledgeUserDataCollection"

        request_body = {
            "acknowledgement": acknowledgement
        }

        headers = {
            "Content-Type": "application/json"
        }

        response = authed_session.post(api_url, headers=headers, data=json.dumps(request_body))

        if response.status_code == 200:
            print("✅ Successfully acknowledged user data collection!")
        else:
            print(f"❌ Error acknowledging user data collection. Status Code: {response.status_code}")
            print(f"Error Response: {response.text}")

    except Exception as e:
        print(f"An unexpected error occurred during user data collection acknowledgment: {e}")


🔼 Upload your client_secrets.json file


Saving client_secrets.json to client_secrets.json
🔗 Open this link in a new tab and authorize:
https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=829270404086-9rr5lh3i3lujoj8jds6oug2pt6v9fdcq.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fanalytics.edit+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fanalytics.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fanalytics.provision&state=KmHrVYgWmAnQRSm8YgQNgujG3pOSip&prompt=consent&access_type=offline
Paste the authorization code here: 4/1AVMBsJiVDfQIoW1kLLBA8f5jn2dbUrz1peeKqhR7z4mNoSKxdl7UsUGhFk8
✅ Authorized GA4 Admin API + Google Sheets successfully!


In [15]:
#STEP 2: Connect to Sheet and Read Config
gsheet_url = input("Paste your Google Sheet URL: ")

try:
    spreadsheet = gc.open_by_url(gsheet_url)
except Exception as e:
    print("❌ ERROR: Could not open spreadsheet. Ensure Google Sheets API is enabled:")
    print("👉 https://console.developers.google.com/apis/api/sheets.googleapis.com/overview")
    raise e

config_sheet = spreadsheet.worksheet("config")
config_values = config_sheet.get_all_values()
headers = [h.strip().lower().replace(" ", "_").replace(")", "") for h in config_values[0]]
data = config_values[1]
config_dict = dict(zip(headers, data))

account_id = config_dict['account_id']
property_name = config_dict['property_name']
time_zone = config_dict['time_zone']
currency = config_dict['currency']
industry_raw = config_dict['industry_categry']

valid_time_zones = ['Asia/Kolkata', 'America/New_York', 'Europe/London', 'Asia/Tokyo', 'Australia/Sydney']
valid_currencies = ['USD', 'INR', 'EUR', 'JPY', 'AUD']
official_industries = [
    "ARTS_AND_ENTERTAINMENT", "AUTOMOTIVE", "BEAUTY_AND_FITNESS", "BOOKS_AND_LITERATURE",
    "BUSINESS_AND_INDUSTRIAL", "BUSINESS_AND_INDUSTRIAL_MARKETS", "COMPUTERS_AND_ELECTRONICS",
    "FINANCE", "FOOD_AND_DRINK", "GAMES", "HEALTHCARE", "HEALTH", "HOME_AND_GARDEN",
    "INTERNET_AND_TELECOM", "JOBS_AND_EDUCATION", "LAW_AND_GOVERNMENT", "NEWS", "ONLINE_COMMUNITIES",
    "PEOPLE_AND_SOCIETY", "PETS_AND_ANIMALS", "PROPERTY", "REFERENCE", "SCIENCE", "SHOPPING",
    "SPORTS", "TECHNOLOGY", "TRAVEL", "OTHER"
]

industry_map = {
    "RETAIL": "BUSINESS_AND_INDUSTRIAL_MARKETS",
    "FINANCE": "FINANCE",
    "HEALTH": "HEALTHCARE",
    "TECH": "TECHNOLOGY",
    "AUTO": "AUTOMOTIVE",
    "TRAVEL": "TRAVEL",
    "OTHER": "OTHER"
}

industry_key = industry_raw.upper().strip()
industry = industry_map.get(industry_key, industry_key)
if industry not in official_industries:
    print(f"⚠️ Invalid industry '{industry}', defaulting to 'OTHER'")
    industry = "OTHER"

if time_zone not in valid_time_zones:
    print(f"⚠️ Invalid time zone '{time_zone}', defaulting to 'Asia/Kolkata'")
    time_zone = 'Asia/Kolkata'

if currency not in valid_currencies:
    print(f"⚠️ Invalid currency '{currency}', defaulting to 'USD'")
    currency = 'USD'

print("\n📋 Loaded Config:")
print(f"- Account ID: {account_id}")
print(f"- Property Name: {property_name}")
print(f"- Time Zone: {time_zone}")
print(f"- Currency: {currency}")
print(f"- Industry: {industry}")

property_id = None

Paste your Google Sheet URL: https://docs.google.com/spreadsheets/d/1SiY9fQk9AX92bqW7NzrCqvu4UKlbt1U6Fqzh1EcVPnw/edit?gid=0#gid=0

📋 Loaded Config:
- Account ID: 356178572
- Property Name: Final Gsheet Test GA4
- Time Zone: Asia/Kolkata
- Currency: INR
- Industry: AUTOMOTIVE


In [16]:
#STEP 3: Create GA4 Property
property_body = {
    "displayName": property_name,
    "timeZone": time_zone,
    "currencyCode": currency,
    "industryCategory": industry,
    'parent':f"accounts/{account_id}",
}

try:
    #Check for existing property with the same display name
    print(f"\n🔍 Checking for existing property named '{property_name}' under account '{account_id}'...")
    existing_properties_list_response = admin.properties().list(filter=f"parent:accounts/{account_id}").execute()
    existing_properties = existing_properties_list_response.get('properties', [])

    property_already_exists = False
    existing_property_name_resource = None
    for prop in existing_properties:
        if prop.get('displayName') == property_name:
            property_already_exists = True
            existing_property_name_resource = prop.get('name')
            break

    if property_already_exists:
        print(f"⚠️ A GA4 Property named '{property_name}' already exists: {existing_property_name_resource}")
        property_id = existing_property_name_resource
    else:
        property_response = admin.properties().create(
            body=property_body
        ).execute()
        property_id = property_response["name"]
        print(f"\n✅ Created GA4 Property: {property_id}")

except Exception as e:
    print(f"\n❌ Error creating property: {e}")
    raise e

if property_id is None:
    raise Exception("GA4 Property ID could not be determined. Cannot proceed with further configurations.")

#STEP 4: Acknowledge User Data Collection
acknowledge_user_data_collection(creds, property_id, ACKNOWLEDGEMENT_STRING)


🔍 Checking for existing property named 'Final Gsheet Test GA4' under account '356178572'...
⚠️ A GA4 Property named 'Final Gsheet Test GA4' already exists: properties/495463367
✅ Successfully acknowledged user data collection!


In [5]:
#STEP 5: Create Custom Dimensions
custom_dims = spreadsheet.worksheet("custom_dimensions").get_all_records()

for dim in custom_dims:
    try:
        admin.properties().customDimensions().create(
            parent=property_id,
            body={
                "parameterName": dim['parameterName'],
                "displayName": dim['displayName'],
                "scope": dim['scope']
            }
        ).execute()
        print(f"✅ Created Custom Dimension: {dim['displayName']}")
    except Exception as e:
        print(f"❌ Error creating dimension {dim['parameterName']}: {e}")

✅ Created Custom Dimension: Click Text
✅ Created Custom Dimension: Loyalty Tier
✅ Created Custom Dimension: Menu Name
✅ Created Custom Dimension: Click To Call


In [6]:
#STEP 6: Create and Configure Key Events
key_events = spreadsheet.worksheet("key_events").get_all_records()

for ev in key_events:
    try:
        key_event = admin.properties().conversionEvents().create(
            parent=property_id,
            body={"eventName": ev['eventName']}
        ).execute()
        print(f"✅ Created Key Event: {ev['eventName']}")

        admin.properties().conversionEvents().patch(
            name=key_event['name'],
            updateMask="counting_method",
            body={"countingMethod": ev['countingMethod']}
        ).execute()
        print(f"    ↳ Counting Method Set: {ev['countingMethod']}")
    except Exception as e:
        print(f"❌ Failed to set key event {ev['eventName']}: {e}")

✅ Created Key Event: loyalty_tier
    ↳ Counting Method Set: ONCE_PER_EVENT
✅ Created Key Event: click_to_call
    ↳ Counting Method Set: ONCE_PER_SESSION


In [11]:
#STEP 7: Update Data Retention Settings
print("\n⚙️ Configuring Data Retention Settings...")

while True:
    retention_input = input("Enter data retention duration in months (2, 14, 26*, 38*, 50*): ")
    try:
        months = int(retention_input)
        if months == 2:
            event_retention = "TWO_MONTHS"
            break
        elif months == 14:
            event_retention = "FOURTEEN_MONTHS"
            break
        elif months in [26, 38, 50]:
            event_retention = f"{months}_MONTHS".upper()
            print(f"❕ Note: Retention periods of {months} months are typically available for GA360 properties.")
            break
        else:
            print("Invalid input. Please enter 2, 14, 26, 38, or 50.")
    except ValueError:
        print("Invalid input. Please enter a number.")

reset_on_new_activity_input = input("Reset user data retention period on new activity? (yes/no, default: yes): ").lower().strip()
reset_on_new_activity = reset_on_new_activity_input == 'yes'

data_retention_body = {
    "eventDataRetention": event_retention,
    "resetUserDataOnNewActivity": reset_on_new_activity
}

try:
    data_retention_settings_name = f"{property_id}/dataRetentionSettings"

    updated_retention_settings = admin.properties().updateDataRetentionSettings(
        name=data_retention_settings_name,
        updateMask="eventDataRetention,resetUserDataOnNewActivity",
        body=data_retention_body
    ).execute()

    print(f"✅ Data Retention Settings Updated for {property_id}:")
    print(f"    - Event Data Retention: {updated_retention_settings.get('eventDataRetention')}")
    print(f"    - Reset on New Activity: {updated_retention_settings.get('resetUserDataOnNewActivity')}")

except Exception as e:
    print(f"❌ Error updating data retention settings: {e}")



⚙️ Configuring Data Retention Settings...
Enter data retention duration in months (2, 14, 26*, 38*, 50*): 14
Reset user data retention period on new activity? (yes/no, default: no): yes
✅ Data Retention Settings Updated for properties/495463367:
    - Event Data Retention: FOURTEEN_MONTHS
    - Reset on New Activity: True


In [12]:
#STEP 8: Reporting Identity Manual Step (Renumbered from 6)
print("\n⚠️ Manual Step Required: Set Reporting Identity in GA4 UI")
print(f"👉 Visit: https://analytics.google.com/analytics/web/#/{account_id}p{property_id.replace('properties/', '')}/admin/datastreams/reporting-identity")


⚠️ Manual Step Required: Set Reporting Identity in GA4 UI
👉 Visit: https://analytics.google.com/analytics/web/#/356178572p495463367/admin/datastreams/reporting-identity


In [14]:
#STEP 9: Summary Display (Renumbered from 7)
try:
    #1. Get GA4 Property Summary
    prop = admin.properties().get(name=property_id).execute()
    print("\n📊 GA4 Property Summary:")
    print(f"Name: {prop.get('displayName')}")
    print(f"Time Zone: {prop.get('timeZone')}")
    print(f"Currency: {prop.get('currencyCode')}")
    print(f"Industry: {prop.get('industryCategory')}")
    print(f"Reporting Identity: Manual")  # Static unless retrieved separately

    #2. Get Data Retention Settings
    current_retention_settings = admin.properties().getDataRetentionSettings(
        name=f"{property_id}/dataRetentionSettings"
    ).execute()

    print("\n⏱️ Current Data Retention Settings:")
    print(f"    - Event Data Retention: {current_retention_settings.get('eventDataRetention')}")
    print(f"    - Reset on New Activity: {current_retention_settings.get('resetUserDataOnNewActivity')}")

    #3. Get Conversion Events
    convs = admin.properties().conversionEvents().list(parent=property_id).execute()
    print("\n🎯 Key Events:")
    if convs.get('conversionEvents'):
        for ev in convs['conversionEvents']:
            print(f" - {ev['eventName']} (Method: {ev['countingMethod']})")
    else:
        print(" - No Key Events configured.")

    #4. Get Custom Dimensions
    dims = admin.properties().customDimensions().list(parent=property_id).execute()
    print("\n🧩 Custom Dimensions:")
    if dims.get('customDimensions'):
        for dim in dims['customDimensions']:
            print(f" - {dim['displayName']} ({dim['parameterName']}, Scope: {dim['scope']})")
    else:
        print(" - No Custom Dimensions configured.")

except Exception as e:
    print(f"❌ Failed to fetch property summary: {e}")


📊 GA4 Property Summary:
Name: Final Gsheet Test GA4
Time Zone: Asia/Calcutta
Currency: INR
Industry: AUTOMOTIVE
Reporting Identity: Manual

⏱️ Current Data Retention Settings:
    - Event Data Retention: FOURTEEN_MONTHS
    - Reset on New Activity: True

🎯 Key Events:
 - click_to_call (Method: ONCE_PER_SESSION)
 - loyalty_tier (Method: ONCE_PER_EVENT)

🧩 Custom Dimensions:
 - Loyalty Tier (loyalty_tier, Scope: USER)
 - Click To Call (click_to_call, Scope: EVENT)
 - Menu Name (menu_name, Scope: EVENT)
 - Click Text (click_text, Scope: EVENT)
